# Download dataset

In [0]:
from urllib.request import urlopen
from shutil import copyfileobj
# https://stackoverflow.com/a/74032774/7876126

my_url = '''https://drive.google.com/uc?export=download&id=1ZJksDgxUTp3dG2GOPlB2vvz3UPknil6_&confirm=t&uuid=68c9d3ff-81ed-48bf-96fa-69254cfae279&at=AKKF8vy3IYHK4s-kB_NS6s58mohz:1688974748736'''
my_filename = "yelp_data.zip"

with urlopen(my_url) as in_stream, open(my_filename, 'wb') as out_file:
    copyfileobj(in_stream, out_file)

check local folder and unzip file

In [0]:
%sh ls

__MACOSX
azure
conf
eventlogs
hadoop_accessed_config.lst
logs
preload_class.lst
yelp_data.zip


In [0]:
%sh unzip yelp_data -d yelp_data_unzip

Archive:  yelp_data.zip
   creating: yelp_data_unzip/yelp_data/
  inflating: yelp_data_unzip/__MACOSX/._yelp_data  
  inflating: yelp_data_unzip/yelp_data/.DS_Store  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._.DS_Store  
  inflating: yelp_data_unzip/yelp_data/yelp_academic_dataset_checkin.json  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._yelp_academic_dataset_checkin.json  
  inflating: yelp_data_unzip/yelp_data/yelp_academic_dataset_tip.json  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._yelp_academic_dataset_tip.json  
  inflating: yelp_data_unzip/yelp_data/yelp_academic_dataset_review.json  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._yelp_academic_dataset_review.json  
  inflating: yelp_data_unzip/yelp_data/yelp_academic_dataset_business.json  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._yelp_academic_dataset_business.json  
  inflating: yelp_data_unzip/yelp_data/yelp_academic_dataset_user.json  
  inflating: yelp_data_unzip/__MACOSX/yelp_data/._yelp

In [0]:
%sh ls -lah yelp_data_unzip/*

yelp_data_unzip/__MACOSX:
total 16K
drwxr-xr-x 3 root root 4.0K Jul 10 07:45 .
drwxr-xr-x 4 root root 4.0K Jul 10 07:45 ..
-rwx------ 1 root root  212 Jul  3 02:51 ._yelp_data
drwxr-xr-x 2 root root 4.0K Jul 10 07:47 yelp_data

yelp_data_unzip/yelp_data:
total 8.7G
drwx------ 2 root root 4.0K Jul  3 02:51 .
drwxr-xr-x 4 root root 4.0K Jul 10 07:45 ..
-rw-r--r-- 1 root root 6.1K Jul  3 02:51 .DS_Store
-rw-r--r-- 1 root root 114M Mar 18  2022 yelp_academic_dataset_business.json
-rw-r--r-- 1 root root 274M Mar 18  2022 yelp_academic_dataset_checkin.json
-rw-r--r-- 1 root root 5.0G Mar 18  2022 yelp_academic_dataset_review.json
-rw-r--r-- 1 root root 173M Mar 18  2022 yelp_academic_dataset_tip.json
-rw-r--r-- 1 root root 3.2G Mar 18  2022 yelp_academic_dataset_user.json


In [0]:
%sh rm -rf yelp_data_unzip/__MACOSX

In [0]:
%sh rm -rf yelp_data_unzip/yelp_data/.DS_Store

Prep the raw file and upload to dbfs

In [0]:
%sh pwd

/databricks/driver


In [0]:
file_local_path = "yelp_data_unzip/yelp_data"
dbfs_file_path = "dbfs:/FileStore/yelp_data"
dbutils.fs.mkdirs(dbfs_file_path)


True

In [0]:
dbutils.fs.cp(f"file:/databricks/driver/{file_local_path}", dbfs_file_path, recurse=True)

True

In [0]:
display(dbutils.fs.ls(dbfs_file_path))

path,name,size,modificationTime
dbfs:/FileStore/yelp_data/yelp_academic_dataset_business.json,yelp_academic_dataset_business.json,118863795,1689039922000
dbfs:/FileStore/yelp_data/yelp_academic_dataset_checkin.json,yelp_academic_dataset_checkin.json,286958945,1689039919000
dbfs:/FileStore/yelp_data/yelp_academic_dataset_review.json,yelp_academic_dataset_review.json,5341868833,1689039951000
dbfs:/FileStore/yelp_data/yelp_academic_dataset_tip.json,yelp_academic_dataset_tip.json,180604475,1689039921000
dbfs:/FileStore/yelp_data/yelp_academic_dataset_user.json,yelp_academic_dataset_user.json,3363329011,1689039970000


# Load dbfs raw files

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.master('local').appName("yelp_data").config("test", "forExample").getOrCreate()

In [0]:
print(dbfs_file_path)
df = {}

for datafile in dbutils.fs.ls(dbfs_file_path):
    datafile_name = datafile.path.split('/')[-1].split('.')[0].split('_')[-1]
    df[datafile_name] = spark.read.format('json').option("inferSchema", True).load(datafile.path)
    print(datafile.path)

print("------------------")
print(df.keys())

dbfs:/FileStore/yelp_data
dbfs:/FileStore/yelp_data/yelp_academic_dataset_business.json
dbfs:/FileStore/yelp_data/yelp_academic_dataset_checkin.json
dbfs:/FileStore/yelp_data/yelp_academic_dataset_review.json
dbfs:/FileStore/yelp_data/yelp_academic_dataset_tip.json
dbfs:/FileStore/yelp_data/yelp_academic_dataset_user.json
------------------
dict_keys(['business', 'checkin', 'review', 'tip', 'user'])


In [0]:
for df_name, df_data in df.items():
    print(f"------------ [df_{df_name}] Schema ------------")
    df_data.printSchema()
    print(f"---------------------------------------------")

------------ [df_business] Schema ------------
root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string 

# Data exploration and Cleansing

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window


In [0]:
for df_name in df.keys():
    distinct_cnt = df[df_name].distinct().count()
    cnt = df[df_name].count()
    print(f"[df_{df_name}] Table Duplicates Check:")
    print(f"Distinct Cnt: {distinct_cnt}; Cnt: {cnt}")

[df_business] Table Duplicates Check:
Distinct Cnt: 150346; Cnt: 150346
[df_checkin] Table Duplicates Check:
Distinct Cnt: 131930; Cnt: 131930
[df_review] Table Duplicates Check:
Distinct Cnt: 6990280; Cnt: 6990280
[df_tip] Table Duplicates Check:
Distinct Cnt: 908848; Cnt: 908915
[df_user] Table Duplicates Check:
Distinct Cnt: 1987897; Cnt: 1987897


df_tip -> SCD ii

In [0]:
from datetime import datetime
max_datetime = datetime(9999, 12, 31, 23, 59, 59)

In [0]:
df_tip_SCD2 = df['tip'] \
    .withColumn("tip_row_num", F.row_number().over(Window.partitionBy([F.col('business_id'), F.col('user_id')]).orderBy(F.col('date')))) \
    .withColumn("start_datetime", F.to_timestamp(F.col('date'))) \
    .withColumn("end_datetime_stg", F.lead(F.col('start_datetime'), 1).over(Window.partitionBy([F.col('business_id'), F.col('user_id')]).orderBy(F.col('date'))) + F.expr('INTERVAL -1 seconds')) \
    .withColumn("max_datetime", F.lit(max_datetime).cast(T.TimestampType())) \
    .withColumn("end_datetime", F.when(F.isnull(F.col('end_datetime_stg')), F.col('max_datetime')).otherwise(F.col('end_datetime_stg'))) \
    .withColumn("is_current", F.when(F.isnull(F.col('end_datetime_stg')), F.lit(True)).otherwise(F.lit(False))) \
    .drop('end_datetime_stg', 'max_datetime')

# display(df_tip_SCD2)

In [0]:
df['tip'] = df_tip_SCD2
df['tip'].show()

+--------------------+----------------+-------------------+--------------------+--------------------+-----------+-------------------+-------------------+----------+
|         business_id|compliment_count|               date|                text|             user_id|tip_row_num|     start_datetime|       end_datetime|is_current|
+--------------------+----------------+-------------------+--------------------+--------------------+-----------+-------------------+-------------------+----------+
|---kPU91CF4Lq2-Wl...|               0|2021-09-23 16:58:31|Great food & serv...|St3aCytjxiMN-5OvF...|          1|2021-09-23 16:58:31|9999-12-31 23:59:59|      true|
|--ARBQr1WMsTWiwOK...|               0|2015-08-15 00:22:09|Very chill & nice...|I2XpWCHAom1JRyHXZ...|          1|2015-08-15 00:22:09|9999-12-31 23:59:59|      true|
|--MbOh2O1pATkXa7x...|               0|2013-05-02 04:03:35|they have fat fre...|qi6OJ4D1e_T_AUQGH...|          1|2013-05-02 04:03:35|2013-05-18 03:40:29|     false|
|--MbOh2O1

df_business ->
- df_business
- df_business_categories
- df_business_attributes

df_business_categories

In [0]:
df_business_cat_exploded = df['business'].select(['business_id', 'categories']).withColumn('category', F.explode(F.split(F.col('categories'), ',')))

In [0]:
# Save 
df['business_cat_exploded'] = df_business_cat_exploded
df['business_cat_exploded'].show()

+--------------------+--------------------+--------------------+
|         business_id|          categories|            category|
+--------------------+--------------------+--------------------+
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|             Doctors|
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Traditional Chin...|
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Naturopathic/Hol...|
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|         Acupuncture|
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|    Health & Medical|
|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|       Nutritionists|
|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|    Shipping Centers|
|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|      Local Services|
|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|            Notaries|
|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|     Mailbox Centers|
|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|   Printing Services|
|tUFrWirKiKi_TAnsV...|Department Stores...|   Department Stores|
|tUFrWirKiKi_TAnsV...|Dep

df_business_attributes

In [0]:
df_business_attributes = df['business'].select(['business_id', 'attributes']).filter(F.col('attributes').isNotNull())

In [0]:
# https://medium.com/@thomaspt748/how-to-flatten-json-files-dynamically-using-apache-pyspark-c6b1b5fd4777
from typing import Final, Dict, Tuple
# from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame as SDF
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

def rename_dataframe_cols(df: SDF, col_names: Dict[str, str]) -> SDF:
    """
    rename columns
    """
    return df.select(*[F.col(col_name).alias(col_names.get(col_name, col_name)) for col_name in df.columns])

def update_column_names(df: SDF, index: int) -> SDF:
    df_temp = df
    all_cols = df_temp.columns
    new_cols = dict((column, f"{column}*{index}") for column in all_cols)
    df_temp = df_temp.transform(lambda df_x: rename_dataframe_cols(df_x, new_cols))

    return df_temp

def flatten_json(df_arg: SDF, index: int = 1) -> SDF:
    """
    flatten json in a spark dataframe with recursion
    """
    df = update_column_names(df_arg, index) if index == 1 else df_arg

    fields = df.schema.fields

    for field in fields:
        data_type = str(field.dataType)
        column_name = field.name
        first_10_chars = data_type[:10]

        if first_10_chars == "ArrayType(":
            # Explode Array
            df_temp = df.withColumn(column_name, F.explode_outer(F.col(column_name)))
            return flatten_json(df_temp, index + 1)
        
        elif first_10_chars == "StructType":
            current_col = column_name

            append_str = current_col

            data_type_str = str(df.schema[current_col].dataType)

            df_temp = df.withColumnRenamed(column_name, column_name + "#1") \
                if column_name in data_type_str else df
            current_col = current_col + "#1" if column_name in data_type_str else current_col

            df_before_expanding = df_temp.select(f"{current_col}.*")
            newly_gen_cols = df_before_expanding.columns

            begin_index = append_str.rfind('*')
            end_index = len(append_str)
            level = append_str[begin_index + 1: end_index]
            next_level = int(level) + 1

            custom_cols = dict((field, f"{append_str}->{field}*{next_level}") for field in newly_gen_cols)
            df_temp2 = df_temp.select("*", f"{current_col}.*").drop(current_col)
            df_temp3 = df_temp2.transform(lambda df_x: rename_dataframe_cols(df_x, custom_cols))
            return flatten_json(df_temp3, index + 1)
    return df

In [0]:
df_business_attributes_flatten = flatten_json(df_business_attributes)

In [0]:
# Save
df['business_attributes_flatten'] = df_business_attributes_flatten
df['business_attributes_flatten'].show()

+--------------------+--------------------------------+---------------------------+-----------------------+------------------------+--------------------+---------------------------+--------------------------+---------------------------+--------------------------------------+------------------------------------------+-------------------------------+---------------------------------+----------------------+-------------------------+-----------------------+-----------------------------------+---------------------------+-------------------------+------------------------------+---------------------------+---------------------------+---------------------------------+-------------------------+---------------------+---------------------+--------------------------+---------------------------+------------------------------+---------------------------------+-----------------------------------------+-----------------------------------+----------------------------------------+-----------------------

df_business

In [0]:
# Clean and save
df['business'] = df['business'].drop('attributes', 'categories')
df['business'].show()

+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|         business_id|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|Pns2l4eNsfO8kk83d...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|mpf3x-BjTdTEA3yCZ...|        Affton|{8:0-18:30, 0:0-0...|      1|    38.551126|    -90.335695|       The UPS Store|      63123|          15|  3.0|   MO|
|5255 E Broadway Blvd|tUFrWirKiKi_TAnsV...|        Tucson|{8:0-23:0, 8:0-22...|      0|    32.223236|   -110.880452|         

user table ->
- user
- user_friends_map

In [0]:
df_user_friends = df['user'].select(['user_id', 'friends'])
df_user_friends = df_user_friends.withColumn("friend", F.split(F.col('friends'), ","))
df['user_friends'] = df_user_friends

df['user_friends'].show()

+--------------------+--------------------+--------------------+
|             user_id|             friends|              friend|
+--------------------+--------------------+--------------------+
|qVc8ODYU5SZjKXVBg...|NSCy54eWehBJyZdG2...|[NSCy54eWehBJyZdG...|
|j14WgRoU_-2ZE1aw1...|ueRPE0CX75ePGMqOF...|[ueRPE0CX75ePGMqO...|
|2WnXYQFK0hXEoTxPt...|LuO3Bn4f3rlhyHIaN...|[LuO3Bn4f3rlhyHIa...|
|SZDeASXq7o05mMNLs...|enx1vVPnfdNUdPho6...|[enx1vVPnfdNUdPho...|
|hA5lMy-EnncsH4JoR...|PBK4q9KEEBHhFvSXC...|[PBK4q9KEEBHhFvSX...|
|q_QQ5kBBwlCcbL1s4...|xBDpTUbai0DXrvxCe...|[xBDpTUbai0DXrvxC...|
|cxuxXkcihfCbqt5By...|HDAQ74AEznP-YsMk1...|[HDAQ74AEznP-YsMk...|
|E9kcWJdJUHuTKfQur...|y2GyxJF5VQWohxgw_...|[y2GyxJF5VQWohxgw...|
|lO1iq-f75hnPNZkTy...|tOQDlz36rI__SOsbL...|[tOQDlz36rI__SOsb...|
|AUi8MPWJ0mLkMfwbu...|gy5fWeSv3Gamuq9Ox...|[gy5fWeSv3Gamuq9O...|
|iYzhPPqnrjJkg1JHZ...|Vq4Pc81l6MWTnc-h4...|[Vq4Pc81l6MWTnc-h...|
|xoZvMJPDW6Q9pDAXI...|6tbXpUIU6upoeqWND...|[6tbXpUIU6upoeqWN...|
|vVukUtqoLF5BvH_Vt...|zkK

In [0]:
df['user'].show()

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

review

In [0]:
df['review'].select('business_id','review_id','user_id').count()

6990280

In [0]:
df['review'].select('business_id','review_id','user_id').distinct().count()

6990280

In [0]:
df['review'].show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

checkin

In [0]:
df_checkin = df['checkin'].withColumn("last_date", F.element_at(F.split(F.col('date'), ','), -1))
df['checkin'] = df_checkin
df['checkin'].show()

+--------------------+--------------------+--------------------+
|         business_id|                date|           last_date|
+--------------------+--------------------+--------------------+
|---kPU91CF4Lq2-Wl...|2020-03-13 21:10:...| 2021-11-11 16:23:50|
|--0iUa4sNDFiZFrAd...|2010-09-13 21:43:...| 2014-04-12 23:04:47|
|--30_8IhuyMHbSOcN...|2013-06-14 23:29:...| 2014-08-13 23:20:22|
|--7PUidqRWpRSpXeb...|2011-02-15 17:12:...| 2015-09-27 13:18:32|
|--7jw19RH9JKXgFoh...|2014-04-21 20:42:...| 2021-06-21 19:59:50|
|--8IbOsAAxjKRoYsB...|2015-06-06 01:03:...| 2018-07-12 21:34:36|
|--9osgUCSDUWUkoTL...|2015-06-13 02:00:...| 2019-01-01 01:49:40|
|--ARBQr1WMsTWiwOK...|2014-12-12 00:44:...| 2018-07-22 01:42:39|
|--FWWsIwxRwuw9vIM...|2010-09-11 16:28:...| 2012-09-16 22:21:21|
|--FcbSxK1AoEtEAxO...|2017-08-18 19:43:...| 2022-01-09 18:33:57|
|--LC8cIrALInl2vyo...|2017-01-12 19:10:...| 2021-12-31 15:22:01|
|--MbOh2O1pATkXa7x...|2013-04-21 01:52:...| 2017-12-02 19:57:32|
|--N9yp3ZWqQIm7DqK...|201

# Persist into parquet file on dbfs

In [0]:
display(dbutils.fs.ls('./FileStore'))

path,name,size,modificationTime
dbfs:/FileStore/tables/,tables/,0,1688848707000
dbfs:/FileStore/yelp_data/,yelp_data/,0,1688975443000
dbfs:/FileStore/yelp_data_parquet/,yelp_data_parquet/,0,1689042171000
dbfs:/FileStore/yelp_dataset/,yelp_dataset/,0,1688970415000


In [0]:
dbutils.fs.mkdirs('/FileStore/yelp_data_parquet/')

True

In [0]:
print(df.keys())
for df_name in df:
    print(f'------------------ [ df_{df_name} ] -----------------')
    persisted_file_path = f'dbfs:/FileStore/tables/yelp_PersistedParquet_{df_name}'
    df[df_name].write.mode("overwrite").parquet(persisted_file_path)
    print(f'---------- [ {persisted_file_path} ] ------------')
    df[df_name].show()

dict_keys(['business', 'checkin', 'review', 'tip', 'user', 'business_cat_exploded', 'business_attributes_flatten', 'user_friends'])
------------------ [ df_business ] -----------------
---------- [ dbfs:/FileStore/tables/yelp_PersistedParquet_business ] ------------
+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|         business_id|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|Pns2l4eNsfO8kk83d...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|mpf3x-BjTdT

In [0]:
print("DONE")

DONE
